In [16]:
import pandas as pd
df=pd.read_csv('mon_data.csv')
df.dropna(subset=['text_image'],inplace=True)


filename          0
url             796
description     796
type              0
filepath          0
scraped           0
old_filename    796
old_filepath      0
text_image        0
dtype: int64

In [2]:
data=df.iloc[:,[-1,3]]

In [3]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()



In [4]:
data.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))
data.head()

C:\Users\sadoun\AppData\Local\Temp\ipykernel_9208\4051206825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text_image = df.text_image.apply(lambda x :preprocess_sentence(x))


,text_image,type
0,facture logo joanna binet coubertin paris fact...,0
1,joanna binet coubertin paris facture facture c...,0
2,facture mon entreprise nom societe adresse pos...,0
3,joanna binet coubertin paris facture cendrillo...,0
4,payer ligne facture sfideli . date creation da...,0


In [5]:
from sklearn.model_selection import train_test_split
X_text_train,X_text_test, y_train, y_test=train_test_split(data.text_image,data.type,test_size=0.2,random_state=1234)

In [43]:
import  tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_text_train)

In [44]:
X_train=tokenizer.texts_to_sequences(X_text_train)
X_test=tokenizer.texts_to_sequences(X_text_test)


0

In [8]:
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=100, padding='post', truncating='post')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=100, padding='post', truncating='post')

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout
model=Sequential()
model.add(Embedding(10000, 200))
model.add(GlobalAveragePooling1D())
model.add(Dense(units = 256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(units = 23, activation = "softmax"))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 200)         2000000   
                                                                 
 global_average_pooling1d_9   (None, 200)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_20 (Dense)            (None, 256)               51456     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 23)                5911      
                                                                 
Total params: 2,057,367
Trainable params: 2,057,367
Non-trainable params: 0
____________________________________________

In [41]:
model.compile(loss="sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train,y_train,epochs = 30, batch_size = 32,validation_data = [X_test, y_test])

Epoch 1/30
40/40 [==============================] - 2s 26ms/step - loss: 2.9407 - accuracy: 0.2524 - val_loss: 2.6512 - val_accuracy: 0.2611
Epoch 2/30
40/40 [==============================] - 1s 21ms/step - loss: 2.5071 - accuracy: 0.3538 - val_loss: 2.3051 - val_accuracy: 0.3631
Epoch 3/30
40/40 [==============================] - 1s 21ms/step - loss: 2.1058 - accuracy: 0.4010 - val_loss: 2.0653 - val_accuracy: 0.3854
Epoch 4/30
40/40 [==============================] - 1s 21ms/step - loss: 1.8551 - accuracy: 0.4513 - val_loss: 1.9042 - val_accuracy: 0.4363
Epoch 5/30
40/40 [==============================] - 1s 21ms/step - loss: 1.6658 - accuracy: 0.5136 - val_loss: 1.7719 - val_accuracy: 0.4904
Epoch 6/30
40/40 [==============================] - 1s 21ms/step - loss: 1.4855 - accuracy: 0.5887 - val_loss: 1.6790 - val_accuracy: 0.4936
Epoch 7/30
40/40 [==============================] - 1s 21ms/step - loss: 1.3604 - accuracy: 0.6078 - val_loss: 1.5997 - val_accuracy: 0.5350
Epoch 8/30
40

In [36]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 3ms/step - loss: 0.6813 - accuracy: 0.0223


[0.6812517642974854, 0.02229299396276474]